In [1]:
import numpy as np
import torch
from pkg_resources import packaging
import glob
from tqdm import tqdm

import os
from  tqdm import tqdm
print("Torch version:", torch.__version__)

Torch version: 1.12.1+cu116


In [2]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [3]:
model, preprocess = clip.load("ViT-B/16")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 149,620,737
Input resolution: 224
Context length: 77
Vocab size: 49408


In [38]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [28]:
image_encoder = model.visual
first_two_blocks = torch.nn.Sequential(
    image_encoder.conv1,  # اولین لایه
    image_encoder.ln_pre,  # Normalization قبل از بلاک‌ها
    *image_encoder.transformer.resblocks[:2],  # دو بلاک اول
).cuda().eval()


In [24]:
first_two_blocks

Sequential(
  (0): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
)

In [15]:
clip.tokenize("Hello World!")

tensor([[49406,  3306,  1002,   256, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)

In [11]:
text_tokens = clip.tokenize(["A photo of a live face"]).cuda()
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()

In [12]:
np.save("live.npy" , text_features.cpu().numpy())

In [6]:
import os
import skimage
import IPython.display

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# images in skimage to use and their textual descriptions
descriptions = {
    "spoof": "A photo of a spoof face",
    "live":  "A photo of a live face",
}

In [7]:
oulu_path ="/media/hassan-hossein/B660FEE360FEA8EF/oulu/preposess/*/cropped_*.jpg"
casia_path ="/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/*/cropped_*.jpg"
msu_path ="/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/*/cropped_*.jpg"
replay_path = "/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/*/cropped_*.jpg"



oulu_paths = glob.glob(oulu_path)
msu_paths = glob.glob(msu_path)
casia_paths = glob.glob(casia_path)
replay_paths = glob.glob(replay_path)

print(len(casia_paths), len(oulu_paths), len(msu_paths), len(replay_paths))

train = oulu_paths + msu_paths +replay_paths


22296 136334 15549 61989


In [8]:
len(casia_paths)

22296

In [9]:
casia_paths

['/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0000.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0005.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0010.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0045.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0050.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0055.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0060.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0065.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0070.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casi

In [29]:
for path in tqdm(train, desc="Processing images"):
    img = Image.open(path).convert("RGB")
    img = preprocess(img)
    img = torch.tensor(img).to("cuda").half() 
    with torch.no_grad():
        #image_features = model.encode_image(torch.unsqueeze(img, dim=0)).float()
        print(torch.unsqueeze(img, dim=0).shape)
        image_features = first_two_blocks(torch.unsqueeze(img, dim=0)).float()
        print(image_features.shape)
    #np.save(os.path.dirname(path) + "/" + "CLIP_" + os.path.basename(path) + ".npy", image_features.cpu().numpy())

Processing images:   0%|                             | 0/213872 [00:00<?, ?it/s]/tmp/ipykernel_32754/2468682447.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img).to("cuda").half()
Processing images:   0%|                             | 0/213872 [00:00<?, ?it/s]

torch.Size([1, 3, 224, 224])


RuntimeError: Given normalized_shape=[768], expected input with shape [*, 768], but got input of size[1, 768, 14, 14]

In [10]:
for path in tqdm(casia_paths, desc="Processing images"):
    print(path)
    img = Image.open(path).convert("RGB")
    img = preprocess(img)
    img = torch.tensor(img).cuda()
    with torch.no_grad():
        image_features = model.encode_image(torch.unsqueeze(img, dim=0)).float()
    np.save(os.path.dirname(path) + "/" + "CLIP_" + os.path.basename(path) + ".npy", image_features.cpu().numpy())

Processing images:   0%|                                                                                           | 0/22296 [00:00<?, ?it/s]/tmp/ipykernel_653902/646262376.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img).cuda()


/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0045.jpg


Processing images:   0%|                                                                                  | 17/22296 [00:01<26:50, 13.84it/s]

/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0050.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0055.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0060.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0065.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0070.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0075.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0080.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0040.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0125.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_spoof_26_NM_3/cropped_0085.jpg


Processing images:   0%|▏                                                                                 | 52/22296 [00:01<07:29, 49.45it/s]

...

/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0060.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0065.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0070.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0075.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0080.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0040.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0125.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0085.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0090.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0095.jpg


Processing images: 100%|█████████████████████████████████████████████████████████████████████████████▊| 22256/22296 [02:11<00:00, 175.74it/s]

/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_6/cropped_0010.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0045.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0050.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0055.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0060.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0065.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0070.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0075.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0080.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_7/cropped_0085.jpg


Processing images: 100%|██████████████████████████████████████████████████████████████████████████████| 22296/22296 [02:12<00:00, 168.71it/s]

/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0075.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0080.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0085.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0090.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0095.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0100.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0105.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0110.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0115.jpg
/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_train_spoof_9_NM_8/cropped_0120.jpg


In [16]:
import torch
import torch.nn as nn

class EarlyCLIP(nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.conv1 = original_model.visual.conv1
        self.ln_pre = original_model.visual.ln_pre
        self.class_embedding = original_model.visual.class_embedding
        self.positional_embedding = original_model.visual.positional_embedding
        self.transformer = original_model.visual.transformer.resblocks[:2]

    def forward(self, x):
        x = self.conv1(x)
        x = x.reshape(x.shape[0], x.shape[1], -1)
        x = x.permute(0, 2, 1)
        x = torch.cat(
            [self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x],
            dim=1
        )
        x = x + self.positional_embedding.to(x.dtype)
        x = self.ln_pre(x)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        return x.mean(dim=0)


In [17]:
first_two_blocks = EarlyCLIP(model)

In [18]:
for path in tqdm(train, desc="Processing images"):
    img = Image.open(path).convert("RGB")
    img = preprocess(img)
    img = torch.tensor(img).to("cuda").half() 
    with torch.no_grad():
        #image_features = model.encode_image(torch.unsqueeze(img, dim=0)).float()
        print(torch.unsqueeze(img, dim=0).shape)
        image_features = first_two_blocks(torch.unsqueeze(img, dim=0)).float()
        print(image_features.shape)
    #np.save(os.path.dirname(path) + "/" + "CLIP_" + os.path.basename(path) + ".npy", image_features.cpu().numpy())

Processing images:   0%|                             | 0/213872 [00:00<?, ?it/s]/tmp/ipykernel_43199/2468682447.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img).to("cuda").half()
Processing images:   0%|                   | 77/213872 [00:00<09:17, 383.60it/s]

torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])


Processing images:   0%|                  | 162/213872 [00:00<08:49, 403.28it/s]

torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])


Processing images:   0%|                  | 250/213872 [00:00<08:25, 422.35it/s]

...

Processing images:   0%|                  | 974/213872 [00:02<08:33, 414.88it/s]

torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])


Processing images:   0%|                 | 1007/213872 [00:02<08:36, 411.79it/s]


torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])
torch.Size([1, 768])
torch.Size([1, 3, 224, 224])


KeyboardInterrupt: 